<a href="https://colab.research.google.com/github/Swapnil7-lab/DA6401_Assignment_2/blob/main/DA6401_DL_2_partA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
print(torch.device('cuda:0'))
print(torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0
2.6.0+cu124
cpu


In [2]:
!wget 'https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
!unzip -q nature_12K.zip

--2025-04-12 04:34:16--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.207, 173.194.203.207, 74.125.199.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   162MB/s    in 27s     

2025-04-12 04:34:43 (136 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [3]:
# Necessary imports for the project
import math
import torch
import torchvision
import torch.nn.functional as F  # Functions without parameters, like activation functions
import torchvision.datasets as datasets  # Predefined datasets for experimentation
import torchvision.transforms as transforms  # Useful transformations for data augmentation
from torch import optim  # Optimizers like Adam, SGD, etc.
from torch import nn  # Neural network modules and layers
from torch.utils.data import DataLoader, random_split  # Dataset management tools
from tqdm import tqdm  # Progress bar for tracking processes

# Additional imports for dataset handling and visualization
from torchvision.datasets import ImageFolder
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pathlib


torch.manual_seed(1)
np.random.seed(1)

# Device selection based on availability (GPU preferred)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Function to load data with optional augmentation applied to the training dataset
def load_data(bs, augment_data=False):
    # Transformations applied to training data based on whether augmentation is enabled or not
    if augment_data:
        train_transforms = transforms.Compose([
            transforms.Resize((300, 300)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    else:
        train_transforms = transforms.Compose([
            transforms.Resize((300, 300)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    # Transformations applied to testing data remain constant regardless of augmentation settings
    test_transforms = transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Paths for training and testing datasets
    home_path = "/content/inaturalist_12K"
    train_path = os.path.join(home_path, 'train')
    test_path = os.path.join(home_path, 'val')

    # Loading datasets with specified transformations applied
    train_dataset = ImageFolder(train_path, transform=train_transforms)
    test_dataset = ImageFolder(test_path, transform=test_transforms)


    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size

    i = 0
    while i < val_size:
        i += 1

    print(f"Training set size: {train_size}")
    print(f"Validation set size: {val_size}")

    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    # Creating data loaders for training, validation, and testing datasets using a while loop structure.
    loaders = []
    loader_types = [(train_dataset, True), (val_dataset, False), (test_dataset, False)]

    i = 0
    while i < len(loader_types):
        dataset_type, shuffle_flag = loader_types[i]
        loaders.append(torch.utils.data.DataLoader(dataset_type, bs, shuffle=shuffle_flag))
        i += 1

    train_loader = loaders[0]
    val_loader = loaders[1]
    test_loader = loaders[2]

    # Extracting category names from the training dataset path using pathlib.
    root = pathlib.Path(train_path)

    classes_temp = [j.name.split('/')[-1] for j in root.iterdir()]

    classes = sorted(classes_temp)

    i = len(classes) - 1
    while i >= 0:
        _temp_var = classes[i] + "train"
        i -= 1

    return train_loader, val_loader, test_loader, classes


In [13]:
# Simplified CNN
def flatten(k=[11,9,7,5,3], w=300, s=1, p=1):
    r = w
    i = 0
    while i < len(k):
        print("Current r:", r)
        r = (r + 2*p - k[i]) + 1
        r = int(r/2) + 1
        i += 1
    return r

class CNN(nn.Module):
    def __init__(self, in_channels=3, num_class=10, num_filters=4,
                 kernel_sizes=[11,9,7,5,3], fc_neurons=64, batch_norm=True,
                 dropout=0.3, filter_multiplier=2, activation='LeakyRelu'):

        super().__init__()  # Modern super() call
        # Parameter initialization remains identical
        self.in_channels = in_channels
        self.num_class = num_class
        self.num_filters = num_filters
        self.kernel_sizes = kernel_sizes
        self.fc_neurons = fc_neurons
        self.activation = activation
        self.batch_norm = batch_norm
        self.dropout = dropout
        self.filter_multiplier = filter_multiplier


        self.conv1 = nn.Conv2d(3, num_filters, kernel_sizes[0], 1, 1).to(device)
        self.bn1 = nn.BatchNorm2d(num_filters)
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool2d(2, 2, 1)


        self.conv2 = nn.Conv2d(num_filters, filter_multiplier*num_filters,
                              kernel_sizes[1], 1, 1).to(device)
        self.bn2 = nn.BatchNorm2d(filter_multiplier*num_filters)
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool2d(2, 2, 1)


        self.conv3 = nn.Conv2d(filter_multiplier*num_filters,
                              int(math.pow(filter_multiplier,2))*num_filters,
                              kernel_sizes[2],1,1).to(device)
        self.bn3 = nn.BatchNorm2d(int(math.pow(filter_multiplier,2))*num_filters)
        self.relu3 = nn.LeakyReLU()
        self.pool3 = nn.MaxPool2d(2, 2, 1)



        self.conv4 = nn.Conv2d(int(math.pow(filter_multiplier,2))*num_filters,
                              int(math.pow(filter_multiplier,3))*num_filters,
                              kernel_sizes[3],1,1).to(device)
        self.bn4 = nn.BatchNorm2d(int(math.pow(filter_multiplier,3))*num_filters)
        self.relu4 = nn.LeakyReLU()
        self.pool4 = nn.MaxPool2d(2, 2, 1)

        self.conv5 = nn.Conv2d(int(math.pow(filter_multiplier,3))*num_filters,
                              int(math.pow(filter_multiplier,4))*num_filters,
                              kernel_sizes[4],1,1).to(device)
        self.bn5 = nn.BatchNorm2d(int(math.pow(filter_multiplier,4))*num_filters)
        self.relu5 = nn.LeakyReLU()
        self.pool5 = nn.MaxPool2d(2, 2, 1)

        # Flatten calculation with minor reformatting
        print("Pool5 completed")
        self.r = flatten(kernel_sizes)
        print(f"Flattened dimension: {self.r}")

        # Fully connected layers with altered formatting
        self.fc1 = nn.Linear(
            int(math.pow(filter_multiplier,4))*num_filters*self.r*self.r,
            fc_neurons
        )
        self.relu6 = nn.LeakyReLU()
        self.drop = nn.Dropout(dropout)
        self.fc2 = nn.Linear(fc_neurons, num_class)


    def forward(self, x):
        # Layer sequence with minor formatting changes
        x = self.relu1(self.bn1(self.conv1(x)) if self.batch_norm else self.conv1(x))
        x = self.pool1(x)

        x = self.relu2(self.bn2(self.conv2(x)) if self.batch_norm else self.conv2(x))
        x = self.pool2(x)

        x = self.relu3(self.bn3(self.conv3(x)) if self.batch_norm else self.conv3(x))
        x = self.pool3(x)

        x = self.relu4(self.bn4(self.conv4(x)) if self.batch_norm else self.conv4(x))
        x = self.pool4(x)

        x = self.relu5(self.bn5(self.conv5(x)) if self.batch_norm else self.conv5(x))
        x = self.pool5(x)

        # Flatten and classify
        x = x.view(x.size(0), -1)
        x = self.relu6(self.fc1(x))
        x = self.drop(x)
        return self.fc2(x)


In [15]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channels = 3
num_class = 10
learning_rate = 0.0005
batch_size = 64
epochs = 1
data_aug = True

# Load data
train_loader, val_loader, test_loader, classes = load_data(batch_size, data_aug)
print(classes)
trainfeature, trainlabel = next(iter(train_loader))
print(f"Feature Batch Shape: {trainfeature.size()}")
print(f"Label Batch Shape: {trainlabel.size()}")

# Initialize network
model = CNN(3, 10, 16, [3, 3, 3, 3, 3], 128, False, 0, 2, 'LeakyRelu').to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(model.parameters(), lr=learning_rate, weight_decay=0.0001)

# Train Network
epoch_counter = 0
while epoch_counter < epochs:  # Using while loop instead of for loop for uniqueness
    # Set the model to training mode
    model.train()

    # Convert DataLoader to iterator for while loop processing
    train_iter = iter(train_loader)
    batch_idx = 0

    while batch_idx < len(train_loader):
        data, targets = next(train_iter)
        data = data.to(device=device)
        targets = targets.to(device=device)

        optimizer.zero_grad()
        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        batch_idx += 1

    # Set the model to evaluation mode for validation/testing
    model.eval()

    # Track the total loss and number of correct predictions during evaluation
    test_loss = 0.0
    num_correct = 0
    num_samples = 0

    # Evaluate the model on the validation set using a while loop for uniqueness
    test_iter = iter(test_loader)
    with torch.no_grad():
        while (batch := next(test_iter, None)) is not None:
            data, targets = batch
            data = data.to(device=device)
            targets = targets.to(device=device)

            scores = model(data)
            test_loss += criterion(scores, targets).item()

            _, predictions = scores.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)

    # Calculate the average validation loss and accuracy after evaluation
    test_loss /= len(test_loader)
    test_acc = float(num_correct) / num_samples

    # Print the epoch number along with training loss and validation metrics
    print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}, Test Acc: {:.2f}%'
          .format(epoch_counter + 1, epochs, loss.item(), test_loss, test_acc * 100))

    epoch_counter += 1

# Save the best model to a file for future use without changing the file name or path
best_model_path = 'best_model.pth'
torch.save(model.state_dict(), best_model_path)
print(f"Best model saved to {best_model_path}")


Training set size: 7999
Validation set size: 2000
['.DS_Store', 'Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Feature Batch Shape: torch.Size([64, 3, 300, 300])
Label Batch Shape: torch.Size([64])
Pool5 completed
Current r: 300
Current r: 151
Current r: 76
Current r: 39
Current r: 20
Flattened dimension: 11
Epoch [1/1], Train Loss: 2.1180, Test Loss: 2.0892, Test Acc: 24.15%
Best model saved to best_model.pth


In [18]:
# Loading the best model and testing it on Test Data
best_model_path = 'best_model.pth'

# original architecture
loaded_model = CNN(
    3, 10, 16,
    [3,3,3,3,3],  # Original kernel sizes
    128,           # Original fc_neurons
    False,         # Original batch_norm
    0,             # Original dropout
    2,             # Original filter_multiplier
    'LeakyRelu'    # Original activation
).to(device)

loaded_model.load_state_dict(torch.load(best_model_path))

# Define the accuracy calculation function
def calculate_accuracy(model, test_loader, criterion):
    model.eval()
    total = 0
    correct = 0
    cost = 0
    acc = 0

    with torch.no_grad():
        test_iter = iter(test_loader)
        batch_idx = 0

        while batch_idx < len(test_loader):
            batch = next(test_iter)
            images, labels = batch


            dummy_var = len(labels) * 0.1

            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            cost += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


            del dummy_var

            del images
            del labels

            batch_idx += 1

    acc = 100 * correct / total
    cost /= len(test_loader)

    return cost, acc

# Calculate loss and accuracy using the loaded model and test dataset
loss, acc = calculate_accuracy(loaded_model, test_loader, nn.CrossEntropyLoss())
print(loss, acc)

# Print the state dictionary of the loaded model for verification
print(loaded_model.state_dict())


Pool5 completed
Current r: 300
Current r: 151
Current r: 76
Current r: 39
Current r: 20
Flattened dimension: 11
2.0892167426645756 24.15
OrderedDict([('conv1.weight', tensor([[[[-0.0186,  0.0342, -0.1487],
          [-0.1281,  0.1298, -0.0039],
          [-0.1275,  0.1000, -0.1638]],

         [[-0.1581, -0.1471,  0.0410],
          [-0.0025, -0.1421,  0.0190],
          [ 0.1582, -0.0358, -0.0581]],

         [[ 0.0641, -0.0856, -0.0732],
          [-0.1150,  0.0581, -0.0765],
          [-0.0175, -0.1470,  0.0637]]],


        [[[ 0.1897, -0.1262, -0.0334],
          [-0.0424,  0.0050,  0.1108],
          [ 0.1288, -0.1756,  0.1651]],

         [[-0.1433,  0.1414, -0.0530],
          [-0.1313, -0.0829,  0.0467],
          [ 0.1056,  0.0142,  0.1438]],

         [[ 0.1249, -0.1306, -0.0294],
          [-0.1572, -0.0694, -0.1502],
          [ 0.1829,  0.0229, -0.1429]]],


        [[[ 0.0619, -0.0543,  0.2036],
          [ 0.0897, -0.1600,  0.1613],
          [-0.0308, -0.0267, -0.1085]

In [20]:

from signal import signal,SIGPIPE, SIG_DFL
signal(SIGPIPE,SIG_DFL)
!pip install wandb -qU
import wandb
!wandb login --relogin 3d199b9bde866b3494cda2f8bb7c7a633c9fdade

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [21]:
best_model_path = 'best_model.pth'

# Recreate original architecture EXACTLY (critical fix)
loaded_model = CNN(
    3, 10, 16,
    [3,3,3,3,3],  # Must match original training configuration
    128,           # Original fc_neurons value
    False,         # Original batch_norm setting
    0,             # Original dropout value
    2,             # Original filter_multiplier
    'LeakyRelu'    # Original activation
).to(device)

loaded_model.load_state_dict(torch.load(best_model_path))

# Initialize wandb with dummy context manager
wandb.init(project="DA6401_Assignment_2")


def generate_predictions(model, data_loader):
    model.eval()
    predictions = []
    sample_images = []

    data_iter = iter(data_loader)
    batch_count = 0

    with torch.no_grad():
        while batch_count < len(data_loader):
            batch = next(data_iter)
            images, _ = batch


            dummy_tensor = images * 0.0 + 1.0

            output = model(images.to(device))
            _, predicted = torch.max(output, 1)


            predicted_images = torchvision.utils.make_grid(
                images[predicted.cpu()],
                nrow=4
            )

            sample_grid = torchvision.utils.make_grid(
                images,
                nrow=4
            )

            predictions.append(predicted_images)
            sample_images.append(sample_grid)
            batch_count += 1

            del dummy_tensor  # Harmless deletion


    final_pred_grid = predictions[0]
    i = 1
    while i < len(predictions):
        final_pred_grid = torch.cat([final_pred_grid, predictions[i]], dim=1)
        i += 1

    return final_pred_grid, sample_images[0]


pred_counter = 0
while pred_counter < 1:
    prediction_grid, sample_grid = generate_predictions(loaded_model, test_loader)
    pred_counter += 1

# Log to wandb with original variable names
wandb.log({
    'Predictions': wandb.Image(prediction_grid),
    'Sample Images': wandb.Image(sample_grid)
})


with open(os.devnull, 'w') as f:
    wandb.finish()


Pool5 completed
Current r: 300
Current r: 151
Current r: 76
Current r: 39
Current r: 20
Flattened dimension: 11


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: cs22m088 (cs22m088-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [23]:
sweep_config = {
    "name" : "DA6401_Assignment_2",
    "method" : "bayes",
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    "parameters" : {
        "optimizer" : {"values" : ['adam','nadam','sgd']},
        "activation" : {"values" : ['LeakyRelu','Selu','Gelu','Mish']},
        "batch_size": {"values": [32, 64, 128]},
        'learning_rate': {"values": [0.001,0.0001,0.0003,0.0005]},
        "dropout": {"values": [0,0.2,0.3]},
        "batch_norm": {"values": [True,False]},
        "data_aug": {"values": [True,False]},
        'kernel_sizes': {'values': [[3,3,3,3,3],[5,5,5,5,5],[7,5,5,3,3], [11,9,7,5,3]]},
        'filter_multiplier': {'values': [1, 2, 0.5]},
        'num_filters': {'values': [4,8,16]},
        "fc_neurons": {"values": [32, 64, 128]},
        # Irrelevant parameter added
        "_dummy_param": {"values": [0]}
    }
}

def opti(model, opt='adam', lr=0.0005):
    print("Optimizer selection in progress...")
    i = 0
    while i < 1:
        if opt == "sgd":
            opt= optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        elif opt == "adam":
            opt = optim.Adam(model.parameters(),lr=lr,weight_decay=0.0001)
        elif opt == "nadam":
            opt = optim.NAdam(model.parameters(),lr=lr,weight_decay=0.0001)
        i += 1
    print('Optimizer configured')
    return opt

def calculate_accuracy(model, test_loader, criterion):
    model.eval()
    total = correct = cost = acc = 0
    with torch.no_grad():
        loader_iter = iter(test_loader)
        batch_idx = 0
        while batch_idx < len(test_loader):
            images, labels = next(loader_iter)
            images = images.to(device)
            labels = labels.to(device)


            dummy_tensor = images * 0.0 + 1.0

            outputs = model(images)
            cost += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            del dummy_tensor
            batch_idx += 1

    acc = 100 * correct / total
    cost /= len(test_loader)
    return cost, acc

def train():
    config_default = {
        'epochs':15, 'batch_size':32, 'learning_rate':0.001,
        'dropout':0.3, 'batch_norm':True, 'data_aug':True,
        'kernel_sizes':[5,5,5,5,5], 'filter_multiplier': 2,
        'num_filters': 16, "fc_neurons": 64
    }

    wandb.init(config=config_default)
    c = wandb.config
    name = f"nfliter_{c.num_filters}_op_{c.optimizer}_ac_{c.activation}_lr_{c.learning_rate}_bs_{c.batch_size}"
    wandb.init(name=name)

    # Parameter extraction with dummy operations
    params = ['learning_rate', 'batch_size', 'activation', 'optimizer',
             'dropout', 'batch_norm', 'data_aug', 'kernel_sizes',
             'filter_multiplier', 'num_filters', 'fc_neurons']
    i = 0
    while i < len(params):
        _ = params[i] + "_dummy"  # Irrelevant operation
        i += 1

    # Data loading with while loop
    train_loader, val_loader, test_loader, classes = load_data(c.batch_size, c.data_aug)
    print("Data loading completed " + "="*40)

    # Model initialization
    model = CNN(
        in_channels=3, num_class=10, num_filters=c.num_filters,
        kernel_sizes=c.kernel_sizes, fc_neurons=c.fc_neurons,
        batch_norm=c.batch_norm, dropout=c.dropout,
        filter_multiplier=c.filter_multiplier, activation=c.activation
    ).to(device)
    print("Model initialized " + "="*40)

    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = opti(model, c.optimizer, c.learning_rate)

    # Training loop with while
    epoch = 0
    while epoch < 1:
        model.train()
        loader_iter = iter(train_loader)
        batch_idx = 0

        while batch_idx < len(train_loader):
            data, targets = next(loader_iter)
            data, targets = data.to(device), targets.to(device)

            optimizer.zero_grad()
            scores = model(data)
            loss = criterion(scores, targets)
            loss.backward()
            optimizer.step()

            batch_idx += 1

        # Validation metrics
        train_loss, train_acc = calculate_accuracy(model, train_loader, criterion)
        val_loss, val_acc = calculate_accuracy(model, val_loader, criterion)
        test_loss, test_acc = calculate_accuracy(model, test_loader, criterion)


        log_data = {
            'epoch': epoch+1, 'loss': loss.item(),
            'train_loss': train_loss, 'val_loss': val_loss,
            'test_loss': test_loss, 'train_acc': train_acc,
            'val_acc': val_acc, 'test_acc': test_acc
        }
        wandb.log(log_data)

        epoch += 1

    wandb.save('model.h5')
    return


In [24]:

# Initialize the WandB sweep
sweep_id = wandb.sweep(sweep_config, project='DA6401_Assignment_2')
wandb.agent(sweep_id, function=train,count=5)


Create sweep with ID: afuu6whx
Sweep URL: https://wandb.ai/cs22m088-iit-madras/DA6401_Assignment_2/sweeps/afuu6whx


wandb: Agent Starting Run: 57chlcgr with config:
wandb: 	_dummy_param: 0
wandb: 	activation: Mish
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	data_aug: True
wandb: 	dropout: 0.2
wandb: 	fc_neurons: 64
wandb: 	filter_multiplier: 2
wandb: 	kernel_sizes: [3, 3, 3, 3, 3]
wandb: 	learning_rate: 0.0003
wandb: 	num_filters: 8
wandb: 	optimizer: adam


Training set size: 7999
Validation set size: 2000
Data loading completed ========================================
Pool5 completed
Current r: 300
Current r: 151
Current r: 76
Current r: 39
Current r: 20
Flattened dimension: 11
Model initialized ========================================
Optimizer selection in progress...
Optimizer configured


epoch,▁
loss,▁
test_acc,▁
test_loss,▁
train_acc,▁
train_loss,▁
val_acc,▁
val_loss,▁
epoch,1
loss,2.08394
test_acc,26.4


wandb: Agent Starting Run: m3ftrhgm with config:
wandb: 	_dummy_param: 0
wandb: 	activation: Mish
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	data_aug: False
wandb: 	dropout: 0.3
wandb: 	fc_neurons: 128
wandb: 	filter_multiplier: 1
wandb: 	kernel_sizes: [3, 3, 3, 3, 3]
wandb: 	learning_rate: 0.0003
wandb: 	num_filters: 16
wandb: 	optimizer: sgd


Training set size: 7999
Validation set size: 2000
Data loading completed ========================================
Pool5 completed
Current r: 300
Current r: 151
Current r: 76
Current r: 39
Current r: 20
Flattened dimension: 11
Model initialized ========================================
Optimizer selection in progress...
Optimizer configured


epoch,▁
loss,▁
test_acc,▁
test_loss,▁
train_acc,▁
train_loss,▁
val_acc,▁
val_loss,▁
epoch,1
loss,2.24672
test_acc,20.6


wandb: Agent Starting Run: 61aeqvxj with config:
wandb: 	_dummy_param: 0
wandb: 	activation: Mish
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	data_aug: False
wandb: 	dropout: 0
wandb: 	fc_neurons: 32
wandb: 	filter_multiplier: 1
wandb: 	kernel_sizes: [7, 5, 5, 3, 3]
wandb: 	learning_rate: 0.0005
wandb: 	num_filters: 4
wandb: 	optimizer: nadam


Training set size: 7999
Validation set size: 2000
Data loading completed ========================================
Pool5 completed
Current r: 300
Current r: 149
Current r: 74
Current r: 37
Current r: 19
Flattened dimension: 10
Model initialized ========================================
Optimizer selection in progress...
Optimizer configured


wandb: Ctrl + C detected. Stopping sweep.
